In [1]:
'''
另外一个cnntext项目例句:    'DESC:manner How did serfdom develop in and then leave Russia ?' 
转化为：   X= ['manner',' How ',' did ',' serfdom','  develop ',' in ',' and ',' then','  leave','  Russia','  ?']     y= 'desc'
转化为：   X= [234,14,1,41124,1414,1414]     y=5
X转化为   300维度的 wordvector
max_len 表示每个batch X最长的长度

(d,v) * (v,max_len,b) =(d, max_len ,b)

K=(3,4,5)  K表示strid     C_f 表示C_f个filter         #三次不同filter size的卷积操作得到，然后用maxpool得到相同的size=(b,C_f,1,1)。然后concate
(d,max_len,b) 卷积操作_*   C_f个(d, K) =  (max_len-K+1,b,C_f)     K为3/4/5
然后三个(max_len-K+1,b,C_f)对于不同的max_len-K+1分别max_pool，得到三个(b,C_f)  
三个(b,C_f)  concate得到 (b,3*C_f)  
(out_size,3*C_f) * (b,3*C_f) = (out_size,b)

(out_size,b)  与b个y   softmax  得到结果
'''

"\n另外一个cnntext项目例句:    'DESC:manner How did serfdom develop in and then leave Russia ?' \n转化为：   X= ['manner',' How ',' did ',' serfdom','  develop ',' in ',' and ',' then','  leave','  Russia','  ?']     y= 'desc'\n转化为：   X= [234,14,1,41124,1414,1414]     y=5\nX转化为   300维度的 wordvector\nmax_len 表示每个batch X最长的长度\n\n(d,v) * (v,max_len,b) =(d, max_len ,b)\n\nK=(3,4,5)  K表示strid     C_f 表示C_f个filter         #三次不同filter size的卷积操作得到，然后用maxpool得到相同的size=(b,C_f,1,1)。然后concate\n(d,max_len,b) 卷积操作_*   C_f个(d, K) =  (max_len-K+1,b,C_f)     K为3/4/5\n然后三个(max_len-K+1,b,C_f)对于不同的max_len-K+1分别max_pool，得到三个(b,C_f)  \n三个(b,C_f)  concate得到 (b,3*C_f)  \n(out_size,3*C_f) * (b,3*C_f) = (out_size,b)\n\n(out_size,b)  与b个y   softmax  得到结果\n"

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F

dtype = torch.FloatTensor



# 3 words sentences (=sequence_length is 3)
sentences = ["i love you", "he loves me", "she likes baseball", "i hate you", "sorry for that", "this is awful"]
labels = [1, 1, 1, 0, 0, 0]  # 1 is good, 0 is not good.

word_list = " ".join(sentences).split()
word_list = list(set(word_list))
word_dict = {w: i for i, w in enumerate(word_list)}
vocab_size = len(word_dict)   #v=16



print(' word_dict=',word_dict)
print('vocab_size=',vocab_size)        

 word_dict= {'for': 0, 'is': 1, 'sorry': 2, 'loves': 3, 'likes': 4, 'that': 5, 'me': 6, 'love': 7, 'he': 8, 'this': 9, 'awful': 10, 'baseball': 11, 'you': 12, 'hate': 13, 'i': 14, 'she': 15}
vocab_size= 16


In [3]:
# Text-CNN Parameter
embedding_size = 2                #d=2
sequence_length = 3               #m_length=m=3
num_classes = 2                   # 0 or 1   => out_size     
filter_sizes = [2, 2, 2]          # n-gram window  =>
num_filters = 3                   # =>f


inputs = []
targets = []
for sen in sentences:
    inputs.append([word_dict[n] for n in sen.split()])


for out in labels:
    targets.append(out) # To using Torch Softmax Loss function

input_batch = Variable(torch.LongTensor(inputs))  #(b,m)
target_batch = Variable(torch.LongTensor(targets))

print("input=",input_batch.size()) 
print("output=",target_batch.size())


input= torch.Size([6, 3])
output= torch.Size([6])


In [4]:
class TextCNN(nn.Module):
    def __init__(self):
        super(TextCNN, self).__init__()

        self.num_filters_total = num_filters * len(filter_sizes)       #3*out_channel
        self.W = nn.Parameter(torch.empty(vocab_size, embedding_size).uniform_(-1, 1)).type(dtype)      #(v,d)
        self.Weight = nn.Parameter(torch.empty(self.num_filters_total, num_classes).uniform_(-1, 1)).type(dtype) #(out_size,3*out_channel)
        self.Bias = nn.Parameter(0.1 * torch.ones([num_classes])).type(dtype)

    def forward(self, X):                      #(b,m)
        embedded_chars = self.W[X]             # (v,d) * (b,m)=(b,m,d)=(6,3,2)      embeddign
        embedded_chars = embedded_chars.unsqueeze(1)       #(b,m,d)===>(b,1,m,d)    add input_channel=1     [b, input_channel(=1), m, d]

        pooled_outputs = []
        for filter_size in filter_sizes:
                    # conv2d( input_channel(=1), output_channel(=3), (filter_height(=2), filter_width(=d) ), bias_option )
            conv = nn.Conv2d(1, num_filters, (filter_size, embedding_size), bias=True)(embedded_chars)        #embedded_chars:(b,1,m,d) 
            #conv:(b,output_channel,m-filter_height+1,d-filter_width+1)=(6,3,3-2+1,d-d+1)=(6,3,2,1)
                    
                    
            h = F.relu(conv)
            # mp : ((filter_height, filter_width))
            mp = nn.MaxPool2d((sequence_length - filter_size + 1, 1))
            mp=  mp(h) #mp:   (b,ouput_channel,2,1)=======>maxpool====>(b,output_channle,1,1)=(6,3,1,1)

            pooled = mp.permute(0, 3, 2, 1)    #pooled: (b,output_channel,1,1)====>(b,1,1,output_channel)=(6,1,1,3)
            pooled_outputs.append(pooled)

        h_pool = torch.cat(pooled_outputs, len(filter_sizes)) #(b,1,1,output_channel*3)=(6,1,1,9)
        h_pool_flat = torch.reshape(h_pool, [-1, self.num_filters_total])   #(b,3*out_channel)=(6,9)

        model = torch.mm(h_pool_flat, self.Weight) + self.Bias     #(out_size,3*out_channel)* (b,3*out_channel)=(b,out_size)
        return model
        


In [5]:
model = TextCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


# Training
for epoch in range(1):
# for epoch in range(5000):
    optimizer.zero_grad()
    output = model(input_batch) #(b,m)
    print(output.size())
    print(output)
    print(target_batch.size())
    print(target_batch)
    # output : (b,out_size)        target_batch :(b) (LongTensor, not one-hot)
    loss = criterion(output, target_batch)
    if (epoch + 1) % 1000 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))

    loss.backward()
    optimizer.step()


torch.Size([6, 2])
tensor([[-0.2810,  1.1076],
        [-0.5391,  0.1760],
        [-0.3084,  0.6615],
        [-0.2210,  0.6378],
        [-0.3741,  1.1915],
        [-0.2388,  1.0404]])
torch.Size([6])
tensor([ 1,  1,  1,  0,  0,  0])


In [10]:
# Test
test_text = 'me love you'
tests = [[word_dict[n] for n in test_text.split()]]
test_batch = Variable(torch.LongTensor(tests))


# Predict
predict = model(test_batch)
print(predict)



# Predict
predict = model(test_batch).data.max(1, keepdim=True)
print(predict)

# Predict
predict = model(test_batch).data.max(1, keepdim=True)[1]
print(predict)





if predict[0][0] == 0:
    print("'",test_text,"'","is Bad Mean")
else:
    print("'",test_text,"'","is Good Mean")

tensor([[-0.9616,  1.3497]])
(tensor([[ 1.6426]]), tensor([[ 1]]))
tensor([[ 1]])
' me love you ' is Good Mean


In [9]:
# Test
test_text = 'me love you'
tests = [[word_dict[n] for n in test_text.split()]]
test_batch = Variable(torch.LongTensor(tests))

# Predict
predict = model(test_batch).data
print(predict)
# if predict[0][0] == 0:
#     print("'",test_text,"'","is Bad Mean")
# else:
#     print("'",test_text,"'","is Good Mean")

tensor([[-1.0265,  1.5113]])
